### Ερωτηση 2
### 1.

In [2]:
import pandas as pd
import sklearn.utils as utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import numpy as np
import gensim 
import gensim.models
from gensim.models import Word2Vec
import gensim.utils as gutils 
import gensim.downloader as api

In [7]:
df_bus = pd.read_csv("philly_restaurants_categories.csv")
print(df_bus.shape)
df_bus.head()

(951, 2)


,business_id,category
0,MUTTqe8uqyMdBl186RmNeA,Japanese
1,QdN72BWoyFypdGJhhI5r7g,Italian
2,WYYdQDjx-DsCanlP0DpImQ,Japanese
3,O1oZpbZNDMH_gz8DhsZCdA,Burgers
4,dChRGpit9fM_kZK5pafNyA,Burgers


In [8]:
df_rev = pd.read_csv("reviews.csv")
df_rev = df_rev[df_rev.business_id.isin(df_bus.business_id)]
print(df_rev.shape)
df_rev.head(2)

(153412, 9)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
3,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14
10,cvQXRFLCyr0S7EgFb4lZqw,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,5.0,3,1,1,"On a scale of one to things that are awesome, ...",2009-10-14 01:15:04


In [9]:
# --------The sum of reviews for each business seperate
df_rev = pd.read_csv("reviews.csv")
df_rev = df_rev[df_rev.business_id.isin(df_bus.business_id)]
df_rev = df_rev.groupby(by= "business_id")["text"].sum()
reviews_by_business = df_bus.merge(df_rev, on="business_id")
print(reviews_by_business.shape)
reviews_by_business.head()

(951, 3)


,business_id,category,text
0,MUTTqe8uqyMdBl186RmNeA,Japanese,Stopped in to check out this new spot around t...
1,QdN72BWoyFypdGJhhI5r7g,Italian,"This place is top notch, with phenomenal servi..."
2,WYYdQDjx-DsCanlP0DpImQ,Japanese,Delicious and very nicely presented. We had J ...
3,O1oZpbZNDMH_gz8DhsZCdA,Burgers,This Wendy's is the worst Wendy's to go to . T...
4,dChRGpit9fM_kZK5pafNyA,Burgers,I am not known for favorable restaurant review...


Σε αυτό το σημείο ανακατεύουμε τα reviews.

In [10]:
SX = utils.shuffle(reviews_by_business) 
print(SX.shape)
SX.head()

(951, 3)


,business_id,category,text
629,kvad5dDPLZ-wuKJmMIs0Ag,Burgers,"My friend, her baby, and myself came in on a S..."
678,JtDZvV3HddxFnNdrwrWh7g,Italian,"Actually, this is a 4.5 star review. My wife K..."
915,f_bUsM34FGpzECag3Cn8gw,Italian,The food I ordered was great the service howev...
129,TjwWB-ET-qmO2-8bfIHMig,Japanese,"Had lunch here today and really, really enjoye..."
584,AqX_5srdyAS1tNiqC3mDvg,Italian,I was here for the first time yesterday for th...


Στο παρακάτω block παίρνω 5 Training, Testing σύνολα από τα ανακατεμένα δεδομένα, όπου σε κάθε fold αντιστοιχώ τα indices που δόθηκαν από το kfold στις πραγματικές τιμές των δεδομένων. Εδώ  δημιουργώ ένα tf-idf vectorizer και παίρνω τις διανυσματικές τιμές των κειμένων των training και testing συνόλων. Αυτές τις τιμές τις χρησιμοποιώ για να κάνω training σε 3 classifiers. Στο τελευταίο fold, στο Logistic Regressor παίρνω τις 20 λέξεις με το μεγαλύτερο και μικρότερο βάρος που χρησιμοποιούνται για τη κατηγοριοποίηση. Από κάθε fold για κάθε αλγόριθμο αποθηκεύω τις μετρικές accuracy, confusion matrix, precision, recall, F1-measure για να υπολογίσω το μέσο αυτών των μετρικών στα 5 folds για κάθε αλγόριθμο ξεχωριστά.

In [40]:
X = SX.text
y = SX.category

accuracy_per_fold = []
confusion_matrix_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_measure_per_fold = []

kf = KFold(n_splits=5)
for fold_num, (train_index, test_index) in enumerate(kf.split(X)):

    # Initializing the lists with the measurements for all algorithms in the current fold
    accuracy_per_algo = []
    confusion_matrix_per_algo = []
    precision_per_algo = []
    recall_per_algo = []
    f1_measure_per_algo = []

    # Coresponding the indeces to the actual data
    print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Getting the tf-idf representation of the training and testing data of the fold
    vectorizer = TfidfVectorizer(stop_words = "english", max_features= 100, min_df=4,max_df=0.8)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    #-------SVM
    svm_clf = svm.SVC()
    svm_clf.fit(X_train,y_train)
    y_pred = svm_clf.predict(X_test)

    accuracy_per_algo.append(metrics.accuracy_score(y_test, y_pred))
    confusion_matrix_per_algo.append(metrics.confusion_matrix(y_test, y_pred))
    precision_per_algo.append(metrics.precision_score(y_test, y_pred, average=None))
    recall_per_algo.append(metrics.recall_score(y_test, y_pred, average=None))
    f1_measure_per_algo.append(metrics.f1_score(y_test,y_pred, average=None))

    #-------K-NN
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)

    accuracy_per_algo.append(metrics.accuracy_score(y_test, y_pred))
    confusion_matrix_per_algo.append(metrics.confusion_matrix(y_test, y_pred))
    precision_per_algo.append(metrics.precision_score(y_test, y_pred, average=None))
    recall_per_algo.append(metrics.recall_score(y_test, y_pred, average=None))
    f1_measure_per_algo.append(metrics.f1_score(y_test,y_pred, average=None))

#------Logistic Regression
    lr_clf = linear_model.LogisticRegression(solver='lbfgs')
    lr_clf.fit(X_train, y_train)
    y_pred = lr_clf.predict(X_test)
    if fold_num == 4:
        categories = lr_clf.coef_.argsort()
        categories = categories[:,::-1]
        terms = vectorizer.get_feature_names()

        print("------------------------------------\nTop 10 words with the Highest weight in the 5th Fold with the Logistic Regressor Classifier:")
        for i in range(len(categories)):
            print(f"Category {lr_clf.classes_[i]}:")
            str_words = ""
            for j in categories[i, :20]:
                str_words += terms[j] + " "
            print(str_words)

        print("\nTop 10 words with the Lowest weight in the 5th Fold with the Logistic Regressor Classifier:")
        for i in range(len(categories)):
            print(f"Category {lr_clf.classes_[i]}:")
            str_words = ""
            for j in categories[i, -20:]:
                str_words += terms[j] + " "
            print(str_words)
   
    accuracy_per_algo.append(metrics.accuracy_score(y_test, y_pred))
    confusion_matrix_per_algo.append(metrics.confusion_matrix(y_test, y_pred))
    precision_per_algo.append(metrics.precision_score(y_test, y_pred, average=None))
    recall_per_algo.append(metrics.recall_score(y_test, y_pred, average=None))
    f1_measure_per_algo.append(metrics.f1_score(y_test,y_pred, average=None))


    # Saving the measurements from the current fold 
    accuracy_per_fold.append(accuracy_per_algo)
    confusion_matrix_per_fold.append(confusion_matrix_per_algo)
    precision_per_fold.append(precision_per_algo)
    recall_per_fold.append(recall_per_algo)
    f1_measure_per_fold.append(f1_measure_per_algo)
    print(f"Completed Fold_{fold_num+1}\n------------")
    

TRAIN: (760,) TEST: (191,)
Completed Fold_1
------------
TRAIN: (761,) TEST: (190,)
Completed Fold_2
------------
TRAIN: (761,) TEST: (190,)
Completed Fold_3
------------
TRAIN: (761,) TEST: (190,)
Completed Fold_4
------------
TRAIN: (761,) TEST: (190,)
------------------------------------
Top 10 words with the Highest weight in the 5th Fold with the Logistic Regressor Classifier:
Category Burgers:
burger fries sandwich wings line drink inside server special cheesesteak steak waitress actually seated prices awesome half bar eating beer 
Category Italian:
italian pasta pizza wine bread dinner salad street night meat perfectly week atmosphere happy perfect excellent lunch disappointed selection highly 
Category Japanese:
sushi ramen spicy fried shrimp sweet pork soup flavor roll rolls reservation options dishes salmon nwe drinks fish tasty different 

Top 10 words with the Lowest weight in the 5th Fold with the Logistic Regressor Classifier:
Category Burgers:
dessert reservation fried l

Οι λέξεις με το μεγαλύτερο βάρος είναι αυτές που καθορίζουν την κατηγορία και έχουν το μεγαλύτερο βάρος στη κατηγορία που ανήκουν. Λέξεις όπως burger , fries καθορίζουν την κατηγορία Burger; italian, pizza, pasta την κατηγορία Italian και sushi, ramen, rice για την Japanese.<br><br>

Βλέπουμε ότι οι λέξεις με το μικρότερο βάρος σε κάθε κατηγορία είναι κοινές με αυτές άλλων κατηγοριών, κάνοντας τη κατηγοριοποίηση πιο δύσκολη, για αυτό και έχουν το μικρότερο βάρος.

Στο παρακάτω block υπολογίζω τα average measurements σε 5 fold, για κάθε αλγόριθμο σε κάθε κλάση. Συγκεκριμένα, έχουμε τα measurements που είναι για κάθε fold(fold->algo), ενώ εμείς θέλουμε να είναι για κάθε αλγόριθμο algorithm ->fold. 
<br>Για να το πέτυχω αυτό χρησιμοποιώ ένα απλό αλγόριθμο που βρήκα στο stack-overfow και έχω ελέγξει ότι δουλεύει.

In [93]:
algo_list = ["SVM", "K-NN", "Logistic Regression"]

accuracy_per_algo_fold = list(map(list, zip(*accuracy_per_fold)))
accuracy_per_algo_fold = np.array(accuracy_per_algo_fold)

confusion_matrix_per_algo_fold = list(map(list, zip(*confusion_matrix_per_fold)))
confusion_matrix_per_algo_fold = np.array(confusion_matrix_per_algo_fold)

precision_per_algo_fold = list(map(list, zip(*precision_per_fold)))
precision_per_algo_fold = np.array(precision_per_algo_fold)

recall_per_algo_fold = list(map(list, zip(*recall_per_fold)))
recall_per_algo_fold = np.array(recall_per_algo_fold)

f1_measure_per_algo_fold = list(map(list, zip(*f1_measure_per_fold)))
f1_measure_per_algo_fold = np.array(f1_measure_per_algo_fold)


for i, algo in enumerate(algo_list):
    print(f"\n\n-----{algo}------")
    print("Accuracy:", np.mean(accuracy_per_algo_fold[i]))
    print("Average confusion matrix:\n", np.mean(confusion_matrix_per_algo_fold[i], axis=0))
    print(f"            {lr_clf.classes_}")
    print("Precision:", np.mean(precision_per_algo_fold[i].T, axis=1))
    print("Recall:", np.mean(recall_per_algo_fold[i].T, axis=1))
    print("F1-measure:", np.mean(f1_measure_per_algo_fold[i].T, axis=1))



-----SVM------
Accuracy: 0.9179939377238908
Average confusion matrix:
 [[55.8  7.2  0.2]
 [ 5.4 80.2  0.6]
 [ 2.   0.2 38.6]]
            ['Burgers' 'Italian' 'Japanese']
Precision: [0.88372235 0.91552093 0.97876192]
Recall: [0.88369477 0.9300495  0.9455732 ]
F1-measure: [0.88328738 0.92227254 0.96153732]


-----K-NN------
Accuracy: 0.8874951777349132
Average confusion matrix:
 [[52.2 10.4  0.6]
 [ 6.  79.4  0.8]
 [ 2.4  1.2 37.2]]
            ['Burgers' 'Italian' 'Japanese']
Precision: [0.86200539 0.87253116 0.96238599]
Recall: [0.8252643  0.9205566  0.91120176]
F1-measure: [0.84283867 0.89562886 0.9357225 ]


-----Logistic Regression------
Accuracy: 0.9232350509782309
Average confusion matrix:
 [[56.6  6.4  0.2]
 [ 4.8 80.8  0.6]
 [ 2.2  0.4 38.2]]
            ['Burgers' 'Italian' 'Japanese']
Precision: [0.89052615 0.92302459 0.98017136]
Recall: [0.89694362 0.93742817 0.93363805]
F1-measure: [0.89317778 0.92960642 0.95550701]


### 2.
Το παρακάτω Block κάνει το pre-processing που χρειάζεται ώστε τα δεδομένα να έχουν την κατάλληλη μορφή για να χρησιμοποιηθούν από τα embeddings.

In [11]:
train_gsim = [gutils.simple_preprocess(x) for x in SX.text]
train_data_labels = [(x,y) for (x,y) in zip(train_gsim, SX.category) if len(x) > 0]

X_gsim = [x for (x,y) in train_data_labels]
y_gsim = [y for (x,y) in train_data_labels]
print(len(X_gsim), len(y_gsim))

951 951


In [141]:
accuracy_per_fold = []
accuracy_per_fold = []
confusion_matrix_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_measure_per_fold = []

kf = KFold(n_splits=5)
for fold_num, (train_index, test_index) in enumerate(kf.split(X_gsim, y_gsim)):

    # Initializing the lists with the measurements for all algorithms in the current fold
    accuracy_per_emb = []
    confusion_matrix_per_emb = []
    precision_per_emb = []
    recall_per_emb = []
    f1_measure_per_emb = []

    # Coresponding the indeces to the actual data in a more manual way than before.
    print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
 
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    for i in train_index:
        X_train.append(X_gsim[i])
        y_train.append(y_gsim[i])
    for i in test_index:
        X_test.append(X_gsim[i])
        y_test.append(y_gsim[i])

    # Training the CBOW model with the mean values of the word embedings
    cbow_model = gensim.models.Word2Vec(X_train, vector_size = 50, sample=1e-05, min_count = 1, window = 10) 
    X_train_cbow = [np.array([cbow_model.wv[x] for x in y]).mean(axis = 0) for y in X_train]
    X_test_cbow = [np.array([cbow_model.wv[x] for x in y if x in cbow_model.wv]).mean(axis = 0) for y in X_test]
    print("Finished CBOW Training")

    # Training the  Skip-Gram model with the mean values of the word embedings
    skipgram_model = gensim.models.Word2Vec(X_train, vector_size = 50, sample=1e-05, min_count = 1, window = 10, sg=1) 
    X_train_skipgram = [np.array([skipgram_model.wv[x] for x in y]).mean(axis = 0) for y in X_train]
    X_test_skipgram = [np.array([skipgram_model.wv[x] for x in y if x in skipgram_model.wv]).mean(axis = 0) for y in X_test]
    print("Finished Skip-Gram Training")

    # Training the Doc2Vec model
    train_corpus = [gensim.models.doc2vec.TaggedDocument(X_train[i], [i]) for i in range(len(X_train))]
    d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=10)
    d2v_model.build_vocab(train_corpus)
    d2v_model.train(train_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)
    X_train_d2v = [d2v_model.infer_vector(x) for x in X_train]
    X_test_d2v = [d2v_model.infer_vector(x) for x in X_test]
    print("Finished Doc2Vec Training")
    

    for X_train, X_test in [(X_train_cbow, X_test_cbow), (X_train_skipgram, X_test_skipgram), (X_train_d2v, X_test_d2v)]:
        
        accuracy_per_algo = []
        confusion_matrix_per_algo = []
        precision_per_algo = []
        recall_per_algo = []
        f1_measure_per_algo = []
        # SVM
        svm_clf = svm.SVC()
        svm_clf.fit(np.array(X_train), np.array(y_train))
        y_pred = svm_clf.predict(np.array(X_test))

        # Saving the results from the current algorithm
        accuracy_per_algo.append(metrics.accuracy_score(y_test, y_pred))
        confusion_matrix_per_algo.append(metrics.confusion_matrix(y_test, y_pred))
        precision_per_algo.append(metrics.precision_score(y_test, y_pred, average=None))
        recall_per_algo.append(metrics.recall_score(y_test, y_pred, average=None))
        f1_measure_per_algo.append(metrics.f1_score(y_test,y_pred, average=None))

        # K-NN
        knn = KNeighborsClassifier()
        knn.fit(np.array(X_train), np.array(y_train))
        y_pred = knn.predict(np.array(X_test))

        # Saving the results from the current algorithm
        accuracy_per_algo.append(metrics.accuracy_score(y_test, y_pred))
        confusion_matrix_per_algo.append(metrics.confusion_matrix(y_test, y_pred))
        precision_per_algo.append(metrics.precision_score(y_test, y_pred, average=None))
        recall_per_algo.append(metrics.recall_score(y_test, y_pred, average=None))
        f1_measure_per_algo.append(metrics.f1_score(y_test,y_pred, average=None))
        
        # Logistic Regressor
        lr_clf = linear_model.LogisticRegression(solver='lbfgs', max_iter=300)
        lr_clf.fit(np.array(X_train), np.array(y_train))
        y_pred = lr_clf.predict(np.array(X_test))

        # Saving the results from the current algorithm
        accuracy_per_algo.append(metrics.accuracy_score(y_test, y_pred))
        confusion_matrix_per_algo.append(metrics.confusion_matrix(y_test, y_pred))
        precision_per_algo.append(metrics.precision_score(y_test, y_pred, average=None))
        recall_per_algo.append(metrics.recall_score(y_test, y_pred, average=None))
        f1_measure_per_algo.append(metrics.f1_score(y_test,y_pred, average=None))

        # Saving the measurements from the current embedding
        accuracy_per_emb.append(accuracy_per_algo)
        confusion_matrix_per_emb.append(confusion_matrix_per_algo)
        precision_per_emb.append(precision_per_algo)
        recall_per_emb.append(recall_per_algo)
        f1_measure_per_emb.append(f1_measure_per_algo)
        
    # Saving the measurements from the current fold 
    accuracy_per_fold.append(accuracy_per_emb)
    confusion_matrix_per_fold.append(confusion_matrix_per_emb)
    precision_per_fold.append(precision_per_emb)
    recall_per_fold.append(recall_per_emb)
    f1_measure_per_fold.append(f1_measure_per_emb)
    print(f"Completed Fold_{fold_num+1}\n------------")
    

TRAIN: (760,) TEST: (191,)
Finished CBOW Training
Finished Skip-Gram Training
Finished Doc2Vec Training
Completed Fold_1
------------
TRAIN: (761,) TEST: (190,)
Finished CBOW Training
Finished Skip-Gram Training
Finished Doc2Vec Training


c:\Users\Giannis\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Completed Fold_2
------------
TRAIN: (761,) TEST: (190,)
Finished CBOW Training
Finished Skip-Gram Training
Finished Doc2Vec Training
Completed Fold_3
------------
TRAIN: (761,) TEST: (190,)
Finished CBOW Training
Finished Skip-Gram Training
Finished Doc2Vec Training
Completed Fold_4
------------
TRAIN: (761,) TEST: (190,)
Finished CBOW Training
Finished Skip-Gram Training
Finished Doc2Vec Training
Completed Fold_5
------------


Σε αυτό το σημείο για κάθε fold, για κάθε embedding, για κάθε αλγόριθμο έχουμε τις ιδίες μετρικές με το προηγούμενο ερώτημα στη μορφή fold->embedding->algorithm, ενώ εμείς τις θέλουμε στη μορφή algorithm->embedding->fold, για να υπολογίσουμε τις μέσες τιμές κάθε μετρικής για κάθε αλγόριθμο εκπαιδευμένος σε κάθε embedding στα 5 folds. Για να τα φέρουμε σε αυτή τη μορφή(algorithm->embedding->fold) θα πρέπει να εφαρμόσουμε τον αλγόριθμο που χρησιμοποιήσαμε στο προηγούμενο ερώτημα αλλά δυο φορές.

In [142]:
algo_list = ["SVM", "K-NN", "Logistic Regression"]
embedding_list = ["CBOW", "Skip-Gram", "Doc2Vec"]

accuracy_per_algo_fold = list(map(list, zip(*accuracy_per_fold)))
accuracy_per_algo_fold = np.array(accuracy_per_algo_fold)

confusion_matrix_per_algo_fold = list(map(list, zip(*confusion_matrix_per_fold)))
confusion_matrix_per_algo_fold = np.array(confusion_matrix_per_algo_fold)

precision_per_algo_fold = list(map(list, zip(*precision_per_fold)))
precision_per_algo_fold = np.array(precision_per_algo_fold)

recall_per_algo_fold = list(map(list, zip(*recall_per_fold)))
recall_per_algo_fold = np.array(recall_per_algo_fold)

f1_measure_per_algo_fold = list(map(list, zip(*f1_measure_per_fold)))
f1_measure_per_algo_fold = np.array(f1_measure_per_algo_fold)

accuracy_per_emb_algo_fold = []
confusion_matrix_per_emb_algo_fold = []
precision_per_emb_algo_fold = []
recall_per_emb_algo_fold = []
f1_measure_per_emb_algo_fold = []

for i in range(len(embedding_list)):
    list2 = list(map(list, zip(*accuracy_per_algo_fold[i]))) 
    list2= np.array(list2)
    accuracy_per_emb_algo_fold.append(list2)

    list2 = list(map(list, zip(*confusion_matrix_per_algo_fold[i]))) 
    list2= np.array(list2)
    confusion_matrix_per_emb_algo_fold.append(list2)

    list2 = list(map(list, zip(*precision_per_algo_fold[i]))) 
    list2= np.array(list2)
    precision_per_emb_algo_fold.append(list2)

    list2 = list(map(list, zip(*recall_per_algo_fold[i]))) 
    list2= np.array(list2)
    recall_per_emb_algo_fold.append(list2)

    list2 = list(map(list, zip(*f1_measure_per_algo_fold[i]))) 
    list2= np.array(list2)
    f1_measure_per_emb_algo_fold.append(list2)
    

for emb_index, emb in enumerate(embedding_list):
    print(f"\n~~~~~~~~ {emb} ~~~~~~~~")
    for i, algo in enumerate(algo_list):
        print(f"\n----- {algo} ------")
        print("Accuracy:", np.mean(accuracy_per_emb_algo_fold[emb_index][i]))
        print("Average confusion matrix:\n", np.mean(confusion_matrix_per_emb_algo_fold[emb_index][i], axis=0))
        print(f"            {lr_clf.classes_}")
        print("Precision:", np.mean(precision_per_emb_algo_fold[emb_index][i].T, axis=1))
        print("Recall:", np.mean(recall_per_emb_algo_fold[emb_index][i].T, axis=1))
        print("F1-measure:", np.mean(f1_measure_per_emb_algo_fold[emb_index][i].T, axis=1))




~~~~~~~~ CBOW ~~~~~~~~

----- SVM ------
Accuracy: 0.8632956737393223
Average confusion matrix:
 [[51.2 11.8  0.2]
 [ 9.6 76.2  0.4]
 [ 3.6  0.4 36.8]]
            ['Burgers' 'Italian' 'Japanese']
Precision: [0.7938974  0.85957322 0.98468835]
Recall: [0.81347217 0.88297202 0.90080708]
F1-measure: [0.80116724 0.87055587 0.93936379]

----- K-NN ------
Accuracy: 0.8748525764673463
Average confusion matrix:
 [[53.  10.2  0. ]
 [ 9.8 75.8  0.6]
 [ 2.4  0.8 37.6]]
            ['Burgers' 'Italian' 'Japanese']
Precision: [0.8105293  0.87118123 0.98447368]
Recall: [0.84149088 0.87843756 0.92117344]
F1-measure: [0.82404964 0.87457111 0.9504946 ]

----- Logistic Regression ------
Accuracy: 0.8811628547809314
Average confusion matrix:
 [[53.  10.   0.2]
 [ 9.2 76.4  0.6]
 [ 1.4  1.2 38.2]]
            ['Burgers' 'Italian' 'Japanese']
Precision: [0.83161825 0.87218456 0.98080808]
Recall: [0.8389141  0.88541532 0.93573102]
F1-measure: [0.83429921 0.87837168 0.95711688]

~~~~~~~~ Skip-Gram ~~~~~~~~


In [1]:
# path = api.load("word2vec-google-news-300", return_path=True)

In [2]:
# g_model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)  

In [4]:

# train_gmodel = [[g_model[x] for x in y if x in g_model] for y in X_gsim[:800]]
# train_data_labels = [(x,y) for (x,y) in zip(train_gmodel, SX.category[:800]) if len(x) > 0]
# X_train= [np.array(x) for (x,y) in train_data_labels]
# y_train = [y for (x,y) in train_data_labels]

# X_train = [x.mean(axis = 0) for x in X_train]


# test_gmodel = [[g_model[x] for x in y if x in g_model] for y in X_gsim[800:]]
# test_data_labels = [(x,y) for (x,y) in zip(test_gmodel, SX.category[800:]) if len(x) > 0]
# X_test = [np.array(x) for (x,y) in test_data_labels]
# y_test = [y for (x,y) in test_data_labels]

# X_test = [x.mean(axis = 0) for x in X_test]

# lr_clf = linear_model.LogisticRegression(solver='lbfgs', max_iter=300)
# lr_clf.fit(X_train, np.array(y_train))
# lr_clf.score(np.array(X_test),y_test)